In [0]:
# Move the file from Workspace to DBFS
dbutils.fs.cp("file:/Workspace/Shared/employee_data.csv", "dbfs:/FileStore/employee_data.csv")

True

In [0]:
# Load the file from DBFS
df = spark.read.format("csv").option("header", "true").load("/FileStore/employee_data.csv")

In [0]:
#Read the data from the csv file
df_csv=spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/FileStore/employee_data.csv")
df_csv.show()

+----------+-------------+----------+-----------+------+
|EmployeeID|         Name|Department|JoiningDate|Salary|
+----------+-------------+----------+-----------+------+
|      1001|     John Doe|        HR| 2021-01-15| 55000|
|      1002|   Jane Smith|        IT| 2020-03-10| 62000|
|      1003|Emily Johnson|   Finance| 2019-07-01| 70000|
|      1004|Michael Brown|        HR| 2018-12-22| 54000|
|      1005| David Wilson|        IT| 2021-06-25| 58000|
|      1006|  Linda Davis|   Finance| 2020-11-15| 67000|
|      1007| James Miller|        IT| 2019-08-14| 65000|
|      1008|Barbara Moore|        HR| 2021-03-29| 53000|
+----------+-------------+----------+-----------+------+



In [0]:
# Assignment 1: Working with CSV Data (employee_data.csv)
# 1. Load the CSV data
df_employee = spark.read.csv('/FileStore/employee_data.csv', header=True, inferSchema=True)
df_employee.show(10)
df_employee.printSchema()

+----------+-------------+----------+-----------+------+
|EmployeeID|         Name|Department|JoiningDate|Salary|
+----------+-------------+----------+-----------+------+
|      1001|     John Doe|        HR| 2021-01-15| 55000|
|      1002|   Jane Smith|        IT| 2020-03-10| 62000|
|      1003|Emily Johnson|   Finance| 2019-07-01| 70000|
|      1004|Michael Brown|        HR| 2018-12-22| 54000|
|      1005| David Wilson|        IT| 2021-06-25| 58000|
|      1006|  Linda Davis|   Finance| 2020-11-15| 67000|
|      1007| James Miller|        IT| 2019-08-14| 65000|
|      1008|Barbara Moore|        HR| 2021-03-29| 53000|
+----------+-------------+----------+-----------+------+

root
 |-- EmployeeID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- JoiningDate: date (nullable = true)
 |-- Salary: integer (nullable = true)



In [0]:
# 2. Data Cleaning:
# Remove rows where the Salary is less than 55,000.
# Filter the employees who joined after the year 2020.
df_cleaned = df_employee.filter((df_employee['Salary'] >= 55000) & (df_employee['JoiningDate'] > '2020-01-01'))
df_cleaned.show()

+----------+------------+----------+-----------+------+
|EmployeeID|        Name|Department|JoiningDate|Salary|
+----------+------------+----------+-----------+------+
|      1001|    John Doe|        HR| 2021-01-15| 55000|
|      1002|  Jane Smith|        IT| 2020-03-10| 62000|
|      1005|David Wilson|        IT| 2021-06-25| 58000|
|      1006| Linda Davis|   Finance| 2020-11-15| 67000|
+----------+------------+----------+-----------+------+



In [0]:
# 3. Data Aggregation:
# Find the average salary by Department.
df_avg_salary_by_dept = df_cleaned.groupBy('Department').agg({'Salary': 'avg'}).withColumnRenamed('avg(Salary)', 'AvgSalary')
df_avg_salary_by_dept.show()

# Count the number of employees in each Department.
df_count_by_dept = df_cleaned.groupBy('Department').count().withColumnRenamed('count', 'EmployeeCount')
df_count_by_dept.show()




+----------+---------+
|Department|AvgSalary|
+----------+---------+
|        HR|  55000.0|
|   Finance|  67000.0|
|        IT|  60000.0|
+----------+---------+

+----------+-------------+
|Department|EmployeeCount|
+----------+-------------+
|        HR|            1|
|   Finance|            1|
|        IT|            2|
+----------+-------------+



In [0]:
# 4. Write the Data to CSV:
# Save the cleaned data (from the previous steps) to a new CSV file.
df_cleaned.coalesce(1).write.csv('/dbfs/FileStore/cleaned_employee_data.csv', header=True)
